In [113]:
!pip install javalang

In [32]:
import os
import hashlib
import ast       #Library to produce AST and analyze it
import astor     #Library to produce the actual code of an AST
#import javalang
#from javalang.tree import *

In [14]:
hash_dict = {}
filesname = []
file_directories = []

In [15]:
"""
In this part we make a list of the files' name and their own directory in the given directory.
A dictionary is also produced having a hash value of the files name as the key and their actual name as the value.
"""

directory = "/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        hash_dict[ hashlib.sha1(str.encode(filename)).hexdigest() ] = filename 
        filesname.append(filename)
        file_directories.append(f)

In [16]:
file_directories

['/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/oauth.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/timeline.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/basetest_project_root.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/repository.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/test_feeds.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/taskd.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/__init__.py',
 '/home/amirmahdi/projects/Ada

In [38]:
"""
This function accepts the source code as a string. It first parse it to AST, then iterate through AST and selects 
nodes that are representing function definition (FunctionDef)
"""

def extract_functions(source_code):
    tree = ast.parse(source_code)
    functions = []
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            functions.append(node)
    return functions

In [34]:
"""
Creating the func_of_files list which each of its elements it is a list of function nodes of a file in the
directory.
These nodes (subtree of the AST) could be used to creat the actual functions in string by using following code:

source_code = astor.to_source(node)
"""

func_of_files = []
for file_dir in file_directories:
    with open(file_dir, "r") as file:
        source_code = file.read()
    functions = extract_functions(source_code)
    func_of_files.append(functions)

In [37]:

source = astor.to_source(func_of_files[0][0])
print(source)

def dummy_oauths():
    from allura.controllers.rest import Oauth1Validator
    dummy_cons_tok = OAuthConsumerToken(api_key=Oauth1Validator().
        dummy_client, name='dummy client, for oauthlib implementation',
        user_id=None)
    session(dummy_cons_tok).flush(dummy_cons_tok)
    dummy_req_tok = OAuthRequestToken(api_key=Oauth1Validator().
        dummy_request_token, user_id=None, validation_pin='dummy-pin')
    session(dummy_req_tok).flush(dummy_req_tok)
    dummy_access_tok = OAuthAccessToken(api_key=Oauth1Validator().
        dummy_access_token, user_id=None)
    session(dummy_access_tok).flush(dummy_access_tok)

